In [1]:
include("../src/ProcessSequence.jl")

Main.ProcessSequence

In [7]:
project_dir = dirname(@__DIR__)

using JSON: parse

project_json = parse(read(joinpath(project_dir, "project.json"), String))

sample_name = project_json["sample_name"]

output_dir = joinpath(project_dir, "output")

input_dir = joinpath(project_dir, "input")

sample_dir = joinpath(input_dir, sample_name, "input")

"/home/jovyan/ProcessSequence.jl/input/738/input"

## Find raw reads

In [8]:
read_file_paths = ProcessSequence.find_reads(sample_dir)

Walking sample directory...

/home/jovyan/ProcessSequence.jl/input/738/input


Number of fastq files found in directories walked: 2

Number of fastq.gz or fq.gz files found in directories walked: 2


Done at: 2020-12-11T23:13:46.301

Took empty period.



2-element Array{Any,1}:
 "/home/jovyan/ProcessSequence.jl/input/738/input/WGC056190D_combined_R1.fastq.gz"
 "/home/jovyan/ProcessSequence.jl/input/738/input/WGC056190D_combined_R2.fastq.gz"

## Run FastQC

In [ ]:
ProcessSequence.check_sequence(
    Tuple(read_file_paths),
    joinpath(output_dir, string("check_sequence_", sample_name)),
    project_json["n_job"],
)

(2020-12-11T23:13:47.467) Checking sequence ...
`fastqc --threads 2 --outdir /home/jovyan/ProcessSequence.jl/output/check_sequence_738 /home/jovyan/ProcessSequence.jl/input/738/input/WGC056190D_combined_R1.fastq.gz /home/jovyan/ProcessSequence.jl/input/738/input/WGC056190D_combined_R2.fastq.gz`


Started analysis of WGC056190D_combined_R1.fastq.gz
Started analysis of WGC056190D_combined_R2.fastq.gz


## Run MultiQC

Not necessary if fastq files came from same lane and same sequencing run. The more complex the production of the reads was, the more useful MultiQC will be to identify potential lane or batch biases.

In [ ]:
ProcessSequence.check_sequence_bias(sample_name, output_dir)

## Concatenate reads of same strand

In [ ]:
ProcessSequence.concatenate_reads(read_file_paths, sample_name, input_dir)

## Run FastQC

In [ ]:
sample_cat_input_dir = joinpath(input_dir, string(sample_name, "_cat"))

sample_cat_output_dir = joinpath(output_dir, string("check_sequence_", sample_name, "_cat"))

if ispath(sample_cat_input_dir)

    ProcessSequence.check_sequence(
        Tuple((
            joinpath(sample_cat_input_dir, string(sample_name, "_R1.fastq.gz")),
            joinpath(sample_cat_input_dir, string(sample_name, "_R2.fastq.gz")),
        )),
        sample_cat_output_dir,
        project_json["n_job"],
    )
    
else
    
    println("Reads weren't concatenated, no need to run FASTQC again.")
    
end